<a href="https://colab.research.google.com/github/widura26/machine-learning-portfolio/blob/main/Books_recommendation_system_content_based_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import regex
import re
import numpy as np

In [ ]:
countvectorizer = CountVectorizer()
text = ["london london paris", "paris paris london"]
count_matrix = countvectorizer.fit_transform(text)

# count_matrix.toarray()
similarity = cosine_similarity(count_matrix)
similarity

array([[1. , 0.8],
       [0.8, 1. ]])

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/dataset/book-dataset'

# Sistem Rekomendasi
<p>Sistem rekomendasi itu dibagi menjadi dua jenis yaitu content based filtering dan collaborative filtering.</p>

Datasets : [GoodBooks](https://www.kaggle.com/datasets/zygmunt/goodbooks-10k)
Other source :

* [Sklearn.tfidf](https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting)

Consine similiarity bisa menggunakan satu argumen maupun dua argumen. Satu Argumen digunakan ketika

## Content Based Filtering

<span>Content-Based Filtering Flow:</span>

* Ambil data item (punya fitur konten).

* Ubah konten → vektor (Count/TF-IDF).

* Hitung similarity antar item (cosine).

* Buat fungsi rekomendasi (ambil top-N mirip).

* (Opsional) gabungkan dengan feedback user.

In [ ]:
books = pd.read_csv(f'{file_path}/books.csv')
tags = pd.read_csv(f'{file_path}/tags.csv')
book_tags = pd.read_csv(f'{file_path}/book_tags.csv')
ratings = pd.read_csv(f'{file_path}/ratings.csv')
to_read = pd.read_csv(f'{file_path}/to_read.csv')

In [ ]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34252 entries, 0 to 34251
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tag_id    34252 non-null  int64 
 1   tag_name  34252 non-null  object
dtypes: int64(1), object(1)
memory usage: 535.3+ KB


In [ ]:
book_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999912 entries, 0 to 999911
Data columns (total 3 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   goodreads_book_id  999912 non-null  int64
 1   tag_id             999912 non-null  int64
 2   count              999912 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981756 entries, 0 to 981755
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   book_id  981756 non-null  int64
 1   user_id  981756 non-null  int64
 2   rating   981756 non-null  int64
dtypes: int64(3)
memory usage: 22.5 MB


In [ ]:
books[books["book_id"] == 2767052]

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


In [ ]:
spesificGoodBook = book_tags[book_tags["goodreads_book_id"] == 2767052] #goodreads_book_id itu mengacu pada satu buku pada edisi tertentu.
spesificGoodBook['tag_id'].duplicated().any() # (false) Jika bernilai false bisa disimpulkan setiap edisi '2767052' memiliki satu tag

np.False_

In [ ]:
book_tags['tag_id'].duplicated().any()
books.duplicated().any()

np.False_

In [ ]:
spesificBook = books[books["work_id"] == 2792775] #work_id itu mengacu pada satu edition. di satu edition itu ada banyak buku beda edition

In [ ]:
books[books["book_id"] == 41865]

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


In [ ]:
book_tags["count"] = pd.to_numeric(book_tags["count"], errors="coerce").fillna(0).astype(int)

In [ ]:
# book_tags_with_name[book_tags_with_name["goodreads_book_id"] == 41865]
book_tags_with_name = book_tags.merge(tags, on="tag_id", how="left")
book_tags_with_name

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,1,11305,37174,fantasy
2,1,11557,34173,favorites
3,1,8717,12986,currently-reading
4,1,33114,12716,young-adult
...,...,...,...,...
999907,33288638,21303,7,neighbors
999908,33288638,17271,7,kindleunlimited
999909,33288638,1126,7,5-star-reads
999910,33288638,11478,7,fave-author


In [ ]:
book_tag_matrix = book_tags_with_name.pivot_table(
    index="goodreads_book_id",
    columns="tag_name",
    values="count",
    aggfunc="sum",
    fill_value=0
)

# values digunakan ketika dataset memiliki kolom data numerik
# aggfunc digunakan ketika terdapat data duplikat
# fill_value digunakan ketika terdapat data NaN

In [ ]:
# book_tag_matrix.loc[2767052, 'to-read']
book_tag_matrix

tag_name,-,--1-,--10-,--12-,--122-,--166-,--17-,--19-,--2-,--258-,...,漫画,골든,﹏moonplus-reader﹏,ﺭﺿﻮﻯ-عاشور,ﻳﻮﺳﻒ-زيدان,Ｃhildrens,Ｆａｖｏｒｉｔｅｓ,Ｍａｎｇａ,ＳＥＲＩＥＳ,ｆａｖｏｕｒｉｔｅｓ
goodreads_book_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31538647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
31845516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32075671,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
similarity_book_matrix = cosine_similarity(book_tag_matrix)
similarity_book_matrix

array([[1.        , 0.98160309, 0.98547144, ..., 0.91776212, 0.39503984,
        0.95450172],
       [0.98160309, 1.        , 0.9847598 , ..., 0.90982876, 0.40503165,
        0.96647656],
       [0.98547144, 0.9847598 , 1.        , ..., 0.91552098, 0.40237257,
        0.99059918],
       ...,
       [0.91776212, 0.90982876, 0.91552098, ..., 1.        , 0.40736123,
        0.89944595],
       [0.39503984, 0.40503165, 0.40237257, ..., 0.40736123, 1.        ,
        0.41014095],
       [0.95450172, 0.96647656, 0.99059918, ..., 0.89944595, 0.41014095,
        1.        ]])

In [ ]:
def recommendBooks(book_id, top_n=100):
  book_index = book_tag_matrix.index.get_loc(book_id)
  sim_scores = similarity_book_matrix[book_index]
  similar_indices = np.argsort(sim_scores)[::-1]
  similar_indices = [i for i in similar_indices if i != book_index]
  # return book_tag_matrix.index[similar_indices].tolist()
  return books[['book_id','title', 'authors']].iloc[similar_indices]

def bookDetail():
  return recommendBooks(41865)

bookDetail()

,book_id,title,authors
5077,4758093,"L.A. Candy (L.A. Candy, #1)",Lauren Conrad
7521,24807,The Areas of My Expertise: An Almanac of Compl...,John Hodgman
7237,15560,"Ilse Witch (Voyage of the Jerle Shannara, #1)",Terry Brooks
6198,11045709,When She Woke,Hillary Jordan
7415,76527,"The Life and Opinions of Tristram Shandy, Gent...","Laurence Sterne, Melvyn New"
...,...,...,...
9808,25614984,Spark Joy: An Illustrated Master Class on the ...,Marie Kondō
4293,85306,"Hour Game (Sean King & Michelle Maxwell, #2)",David Baldacci
9296,827497,One Flew Over the Cuckoo's Nest,Dale Wasserman
9992,4936457,The Time Traveller's Guide to Medieval England...,Ian Mortimer


## Collaborative Filtering

In [ ]:
ratings[ratings["user_id"] == 1]

,book_id,user_id,rating
117889,1180,1,4
488112,4893,1,3
625717,6285,1,4


In [ ]:
# users_total = ratings.groupby("user_id").count()
# books_total = ratings.groupby("book_id").count()
books_ratings = pd.merge(books, ratings, on="book_id", how="right")

In [ ]:
books_ratings.isna().sum()

,0
id,902055
book_id,0
best_book_id,902055
work_id,902055
books_count,902055
isbn,902934
isbn13,902453
authors,902055
original_publication_year,902155
original_title,903836


In [ ]:
# books_ratings.dropna(inplace=True)
# ratings['rating'] = ratings['rating'].fillna(0)

In [ ]:
# books_ratings = books_ratings.dropna(subset=['user_id', 'book_id'])
# books_ratings['user_id'] = books_ratings['user_id'].astype(int)

In [ ]:
# books_ratings
books_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981756 entries, 0 to 981755
Data columns (total 25 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         79701 non-null   float64
 1   book_id                    981756 non-null  int64  
 2   best_book_id               79701 non-null   float64
 3   work_id                    79701 non-null   float64
 4   books_count                79701 non-null   float64
 5   isbn                       78822 non-null   object 
 6   isbn13                     79303 non-null   float64
 7   authors                    79701 non-null   object 
 8   original_publication_year  79601 non-null   float64
 9   original_title             77920 non-null   object 
 10  title                      79701 non-null   object 
 11  language_code              68562 non-null   object 
 12  average_rating             79701 non-null   float64
 13  ratings_count              79

In [ ]:
user_item_matrix = books_ratings.pivot_table(
    index="user_id",
    columns="book_id",
    values="rating",
    fill_value=0
)

In [ ]:
ratings[ratings["user_id"] == 2]

In [ ]:

user2_ratings = user_item_matrix.loc[2]
rated_books = user2_ratings[user2_ratings != 0]
rated_books

In [ ]:
# books_ratings[books_ratings["user_id"] == 2]
# '''data rating yang dilakukan oleh user id pada tabel ratings tidak sinkron / sesuai dengan user matrix.
# Dikarenakan pada tabel book setelah di merge, terdapat dua dari tiga buku yang tidak tersedia'''

books_ratings2 = pd.merge(books, ratings, on="book_id", how="left")
books_ratings2[books_ratings2["user_id"] == 2]
# Hal tersebut dapat dibuktikan dengan mengecek apakah user id 2 sudah memberikan rating buku berdasarkan id
books_ratings2[(books_ratings2["book_id"] == 9762) & (books_ratings2["user_id"] == 2)]

In [ ]:
user_item_matrix

In [ ]:
user_similarity = cosine_similarity(user_item_matrix)
user_similarity

In [ ]:
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
print(user_similarity_df)

In [ ]:
# sim_scores = user_similarity[2]

In [ ]:
# sim_scores[(sim_scores != 1.0) & (sim_scores != 0.0)]

In [ ]:
books_ratings[books_ratings['user_id'] == 1]

In [ ]:
def predict_rating(user_id, book_id):
    user_idx = user_id - 1
    # sim_scores = user_similarity[user_idx]
    # ratings = user_item_matrix[book_id].values
    # pred = np.dot(sim_scores, ratings) / sim_scores.sum()
    return user_idx

In [ ]:
prediksi = predict_rating(3, 9762)
print(prediksi)

In [ ]:
user_similarity[0]